In [1]:
import sqlite3 as sl
import pandas as pd
import datetime
import numpy as np
import pickle
import random

from binance.client import Client
from binance.enums import *
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from pytrends.request import TrendReq as UTrendReq
from headers_file import header_useragent
header = header_useragent
GET_METHOD='get'
class TrendReq(UTrendReq):
    def _get_data(self, url, method=GET_METHOD, trim_chars=0, **kwargs):
        return super()._get_data(url, method=GET_METHOD, trim_chars=trim_chars, headers=header, **kwargs)

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

import os
import sqlite3 as sl
from sqlite3 import Error

cmc = CoinMarketCapAPI('a01ace6b-f43c-439e-9fca-dc7f73e2a648')
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

def getminutedata(symbol, interval, start, end):
    if(end == 'now'):
        frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    else:
        frame = pd.DataFrame(client.get_historical_klines(symbol, 
                                                        interval, 
                                                        start, 
                                                        end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

def get_trends_3(coin, start, end, sleepval):
    pytrend = TrendReq(hl='en-GB')
    data = pytrend.get_historical_interest(coin, 
                                    year_start=start[3], 
                                    month_start=start[2], 
                                    day_start=start[1], 
                                    hour_start=start[0], 
                                    year_end=end[3], 
                                    month_end=end[2], 
                                    day_end=end[1], 
                                    hour_end=end[0], 
                                    cat=0, 
                                    geo='', 
                                    gprop='', 
                                    sleep=sleepval)
    data = data.drop('isPartial', axis=1)
    return data
    


def get_trends_live(coin):
    headers = header_useragent
    pytrends = TrendReq(hl='en-GB')
    pytrends.build_payload([coin], timeframe='now 7-d')
    trend = pytrends.interest_over_time()
    return trend


In [2]:
def fetch_data(id_list, artifact_table, crypto_table):
    for key, artifact_id in enumerate(id_list):
        value = getminutedata(artifact_table.iloc[artifact_id]['ticker'], 
                                            '15m',
                                            '120',
                                            'now')
        close = value[['Time','Close']]                             
        close.index = pd.to_datetime(close['Time'])
        close = close.drop(columns=['Time'])
        close = close.rename(columns={"Close": f"{artifact_id}"})
        
        trend = get_trends_live(crypto_table.iloc[artifact_table.iloc[artifact_id]['crypto_id']]['name_searchable'])
        trend = trend.drop('isPartial', axis=1)
        trend.index = pd.to_datetime(trend.index)
        trend = trend.rename(index={0: f"{artifact_id}"})
        
        if (key != 0):
            trends = trends.join(trend)
            closes = closes.join(close)
        else:
            trends = trend
            closes = close
    return trends, closes

#threshold = 80
# buy check if latest == 100
# once a buy has triggered wait for sell? dont buy after already previous buy indicator late??

def fetch_signals(buy_threshold, sell_threshold , trends, closes):
    #print(buy_threshold)

    check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100))
    check_non0 = trends.iloc[0:-2].mean() > 7
    check_peak = (trends.iloc[0:-10] < buy_threshold).all()
    upward = closes.diff(periods=6)
    check_inc = (upward.iloc[-1] >= 0)

    indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak, check_inc), dtype=bool)

    check_fall = (trends.iloc[-1] < sell_threshold)
    #indicate_sell = np.logical_and.reduce((check_fall), dtype=bool)
    indicate_sell = check_fall
    return indicate_buy, indicate_sell

In [3]:
con = sl.connect("crypto_database.db")
cur = con.cursor()

# settings = pd.read_pickle("./id_list_20_12_2.pkl")
# settings = settings.set_index('id')
# settings.index = settings.index.astype(int)
    # settings = settings.drop([226, 227])#, 243, 266, 227, 189, 78, 211
# id_list = settings.index.tolist()
# id = [149]

artifactString = 'SELECT * FROM artifact'
artifact_table = pd.read_sql_query(artifactString, con)

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, con)

id_list_pre = artifact_table.id.astype(int).tolist()
id_list_pre.remove(38)
id_list_pre.remove(47)
id_list_pre.remove(49)
id_list_pre.remove(71)
id_list_pre.remove(122)
con.close()

In [4]:
#Settings and setup
id_list = id_list_pre[:150]
maxholds = 3
trades = [[],[],[],[]]
tradedata = []
buyPrice  = [0] * (len(id_list))
sellPrice = [0] * (len(id_list))
tradeBuy  = [0] * (len(id_list))
tradeSell = [0] * (len(id_list))
buyTime   = [0] * (len(id_list))
sellTime  = [0] * (len(id_list))
holding = [False] * (len(id_list))
buy_threshold = 80
sell_threshold = 50

In [5]:
import time
run = 1
while (run == 1):
    ##############             Bot Program             ##############
    # Fetch ID close prices  & trend 
    trends, closes = fetch_data(id_list, artifact_table, crypto_table)
    
    # Fetch ID's for buy & sell indicator
    indicate_buy, indicate_sell = fetch_signals(buy_threshold, sell_threshold, trends, closes)

    #upload indicate buy and trends and closes to database


    buy = np.logical_and( indicate_buy, np.logical_not(holding) )
    buy_ids = [id_list[i] for i in [i for i, x in enumerate(buy) if x]]
    if sum(buy) > 0:
        print([trends.index[-1], buy_ids])
    #print(buy_ids)
    sell = np.logical_and(indicate_sell, holding)
    sell_ids = [id_list[i] for i in [i for i, x in enumerate(sell) if x]]
    #if sum(sell) > 0:
    #    print([trends.index[-1], sell_ids])
    #print(sell_ids)

    # Buy control
    if ((sum(holding) < maxholds) and buy_ids):
        for id in reversed(buy_ids):
            if (sum(holding) < maxholds):
                key = id_list.index(id)
                tradetime = closes.index[-1]
                buyPrice[key] = closes.loc[tradetime,str(id)]
                buyTime[key] = tradetime
                holding[key] = True
                # Buy execute function binance using ticker
            else:
                continue
    # Sell control
    if (sell_ids):
        for id in sell_ids:
            # Sell execute function binance using ticker
            key = id_list.index(id)
            tradetime = closes.index[-1]
            sellPrice[key] = closes.loc[tradetime,str(id)]
            sellTime[key] = tradetime
            tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
            holding[key] = False
    print("Finished cycle " + str(datetime.datetime.now()))
    
    run = 0
    time.sleep(60*30*2)
    # Calculate profit
    # tradedata = np.array(tradedata)
    # scaled = ((tradedata[:,5] - 1) * 1/maxholds) + 1
    # scaled.prod()

    

Finished cycle 2022-02-13 17:43:18.297615


In [6]:
trends

#pd.options.display.max_rows = 4000
#crypto_table.iloc[artifact_table.iloc[4]['crypto_id']]['name_searchable']

#print(  ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100)) )


#(trends.iloc[0:-10] > buy_threshold).any()

    # check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100))
    # check_non0 = trends.iloc[0:-2].mean() > 7
    # check_peak = (trends.iloc[0:-10] > buy_threshold).any()
    # upward = closes.diff(periods=6)
    # check_inc = (upward.iloc[-1] >= 0)

    # indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak, check_inc), dtype=bool)

,1inch Network,Aave,AC Milan Fan Token,Cardano,Ambire AdEx,Adventure Gold,Aion,Akropolis,Algorand,MyNeighborAlice,...,S.S. Lazio,Linear,Chainlink,Litentry,Livepeer,Loopring,Lisk,Litecoin,LTO Network,LUNA coin
date,,,,,,,,,,,,,,,,,,,,,
2022-02-06 18:00:00,51,85,0,64,0,84,68,50,68,0,...,2,43,64,50,8,67,81,59,31,58
2022-02-06 19:00:00,17,87,66,66,0,44,62,35,68,56,...,6,42,52,25,6,63,86,52,63,53
2022-02-06 20:00:00,37,82,0,70,0,25,58,23,80,20,...,10,50,60,36,6,57,57,63,0,48
2022-02-06 21:00:00,20,78,0,68,0,54,66,24,83,43,...,15,49,71,39,14,81,84,60,45,46
2022-02-06 22:00:00,0,100,0,72,48,68,90,17,87,72,...,5,53,64,11,18,69,100,61,10,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-13 13:00:00,0,34,30,41,0,53,61,46,39,16,...,7,38,41,7,12,21,41,39,14,26
2022-02-13 14:00:00,0,36,0,45,0,50,53,50,43,31,...,3,39,48,0,9,20,42,42,6,35
2022-02-13 15:00:00,0,34,0,42,0,31,65,60,52,0,...,3,39,45,21,9,22,63,38,13,43


In [ ]:
    # check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100))
    # check_non0 = trends.iloc[0:-2].mean() > 7
    # check_peak = (trends.iloc[0:-10] < buy_threshold).all()
    # upward = closes.diff(periods=6)
    # check_inc = (upward.iloc[-1] >= 0)

    # indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak, check_inc), dtype=bool)

    # check_fall = (trends.iloc[-1] < sell_threshold)
    # indicate_sell = np.logical_and.reduce((check_fall), dtype=bool)

import schedule

schedule.every().hour.do(check_to_buy, 80)
#schedule.every().hour.do(check_to_buy ,70)
schedule.every().hour.do(check_to_buy, 60)
#schedule.every().hour.do(check_to_buy, 50)
schedule.every().hour.do(check_to_buy, 40)

while True:
    schedule.run_pending()

from apscheduler.schedulers.blocking import BlockingScheduler

def some_job():
    print "Decorated job"

scheduler = BlockingScheduler()
scheduler.add_job(some_job, 'interval', hours=1)
scheduler.start()



import smtplib
from email.message import EmailMessage

def email_alert(subject, body, to):
    msg = EmailMessage()
    msg.set_content(body)
    user = "coinbotalert@gmail.com"
    password = "lrmadfwrsgqnrnla"
    msg['subject'] = subject
    msg['to'] = to
    #msg['from'] = user
    msg['from'] = "coinbotalert@gmail.com"

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)

    server.send_message(msg)
    server.quit()

email_alert("!!! Your subscription to SEX ADDICTS anon has been renewed !!!!", "there is no secret ingredient; it's just you", "craigrbirch@icloud.com")
